In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import os

In [3]:
PROJECT_DIR = os.path.abspath('.')
if PROJECT_DIR.endswith('final-nbs'):
    PROJECT_DIR = os.path.abspath('../')
    os.chdir(PROJECT_DIR)

In [4]:
import cfg
from src.data import get_features_path_from_metadata, join_dataframe_columns
from src import util
from src.data import setup_directories

util.setup_logging()

dirs = setup_directories(cfg.DATA_DIR, create_dirs=True)

In [5]:
raw_dir = Path(dirs['raw'])

In [6]:
# read metadata
pd_metadata = pd.read_csv(raw_dir / "metadata.csv", index_col="sample_id")
pd_metadata.head()

,split,instrument_type,features_path,features_md5_hash
sample_id,,,,
S0000,train,commercial,train_features/S0000.csv,017b9a71a702e81a828e6242aa15f049
S0001,train,commercial,train_features/S0001.csv,0d09840214054d254bd49436c6a6f315
S0002,train,commercial,train_features/S0002.csv,3f58b3c9b001bfed6ed4e4f757083e09
S0003,train,commercial,train_features/S0003.csv,e9a12f96114a2fda60b36f4c0f513fb1
S0004,train,commercial,train_features/S0004.csv,b67603d3931897bfa796ac42cc16de78


In [7]:
train_files = get_features_path_from_metadata(pd_metadata.query("split == 'train'"), raw_dir)
valid_files = get_features_path_from_metadata(pd_metadata.query("split == 'val'"), raw_dir)
test_files = get_features_path_from_metadata(pd_metadata.query("split != 'train'"), raw_dir)

In [8]:
len(train_files), len(valid_files), len(test_files)

(766, 293, 804)

In [9]:
from src.preprocessing import apply_preprocessing_fn

In [10]:
def compute_weighted_mass(sample_data: pd.DataFrame):
    mz_abun_sum = (sample_data.groupby(['m/z'])['abundance']
                   .sum()
                   .to_frame('abun_sum')
                   .reset_index())
    total_abun = mz_abun_sum['abun_sum'].sum()
    
    return mz_abun_sum[['m/z', 'abun_sum']].prod(axis=1).sum() / total_abun
    
    
def compute_sample_features(sample_data: pd.DataFrame) -> pd.DataFrame:
    output = {}
    # molecular ion
    output['mol_ion_less99'] = int(sample_data['m/z'].max() < 99)
    # abundance sum could tell us about
    output['weighted_mass'] = compute_weighted_mass(sample_data)
    output['max_temp'] = sample_data['temp'].max()
    output['min_temp'] = sample_data['temp'].min()
    output['temp_range'] = output['max_temp'] - output['min_temp']
    return pd.DataFrame([output]).add_prefix('sample_')
    

In [11]:
_train_sample_features = apply_preprocessing_fn(train_files, pd_metadata, processing_fn=compute_sample_features)

train_sample_features = pd.concat(
    _train_sample_features, names=['sample_id', 'dummy_index']
    ).reset_index(level='dummy_index', drop=True)

  0%|          | 0/766 [00:00<?, ?it/s]

In [12]:
_valid_sample_features = apply_preprocessing_fn(valid_files, pd_metadata, processing_fn=compute_sample_features)

valid_sample_features = pd.concat(
    _valid_sample_features, names=['sample_id', 'dummy_index']
    ).reset_index(level='dummy_index', drop=True)

  0%|          | 0/293 [00:00<?, ?it/s]

In [13]:
_test_sample_features = apply_preprocessing_fn(test_files, pd_metadata, processing_fn=compute_sample_features)

test_sample_features = pd.concat(
    _test_sample_features, names=['sample_id', 'dummy_index']
    ).reset_index(level='dummy_index', drop=True)

  0%|          | 0/804 [00:00<?, ?it/s]

In [14]:
train_dir = Path(dirs['train'])
valid_dir = Path(dirs['valid'])
test_dir = Path(dirs['test'])

In [15]:
from sklearn.preprocessing import QuantileTransformer


for feature in train_sample_features.columns.drop('sample_mol_ion_less99'):
    quantile_transformer = QuantileTransformer(n_quantiles=50).fit(train_sample_features[[feature]])
    train_sample_features[[feature]] = quantile_transformer.transform(train_sample_features[[feature]])
    valid_sample_features[[feature]] = quantile_transformer.transform(valid_sample_features[[feature]])
    test_sample_features[[feature]] = quantile_transformer.transform(test_sample_features[[feature]])
    

In [16]:
train_sample_features.head()

,sample_mol_ion_less99,sample_weighted_mass,sample_max_temp,sample_min_temp,sample_temp_range
sample_id,,,,,
S0000,0,0.518489,0.005352,0.880821,0.006048
S0001,1,0.469380,0.040891,0.018339,0.041015
S0002,0,0.581730,0.190760,0.078410,0.193533
S0003,0,0.896246,0.482619,0.233902,0.489582
S0004,0,0.549664,0.593211,0.614802,0.586967


In [17]:
valid_sample_features.head()

,sample_mol_ion_less99,sample_weighted_mass,sample_max_temp,sample_min_temp,sample_temp_range
sample_id,,,,,
S0766,0,0.005432,0.501948,0.340250,0.505124
S0767,0,0.065747,0.930877,0.220117,0.934335
S0768,0,0.342944,0.849340,0.737245,0.847321
S0769,0,0.864026,0.081626,0.962396,0.081617
S0770,0,0.765547,0.674757,0.653061,0.675945


In [18]:
test_sample_features.head()

,sample_mol_ion_less99,sample_weighted_mass,sample_max_temp,sample_min_temp,sample_temp_range
sample_id,,,,,
S0766,0,0.005432,0.501948,0.340250,0.505124
S0767,0,0.065747,0.930877,0.220117,0.934335
S0768,0,0.342944,0.849340,0.737245,0.847321
S0769,0,0.864026,0.081626,0.962396,0.081617
S0770,0,0.765547,0.674757,0.653061,0.675945


In [19]:
train_sample_features.to_csv(train_dir / 'sample_features.csv', index=True)

In [20]:
valid_sample_features.to_csv(valid_dir / 'sample_features.csv', index=True)

In [21]:
test_sample_features.to_csv(test_dir / 'sample_features.csv', index=True)

### analysis 

In [22]:
# pd_train_target = pd.read_csv(train_dir / 'multiclass.csv', index_col='sample_id')
# train_sample_features = train_sample_features.join(pd_train_target)

In [23]:
# pd_valid_target = pd.read_csv(train_dir / 'multiclass.csv', index_col='sample_id')
# valid_sample_features = valid_sample_features.join(pd_valid_target)

In [24]:
# train_sample_features['is_sam_testbed'] = train_sample_features.index.map(pd_metadata['instrument_type']) == 'sam_testbed'

# test_sample_features['is_sam_testbed'] = test_sample_features.index.map(pd_metadata['instrument_type']) == 'sam_testbed'

# train_sample_features.groupby(train_sample_features['sample_max_mz'] < 99)['is_sam_testbed'].sum()

# test_sample_features.groupby(test_sample_features['sample_max_mz'] < 99)['is_sam_testbed'].sum()

# for target_name in pd_train_target.columns:
    
#     display((train_sample_features.groupby(train_sample_features['sample_max_mz'] < 99)
#      [target_name]
#      .value_counts()
#      .to_frame('count')
#      .sort_index() 
#     ))

# train_sample_features.groupby(train_sample_features['sample_max_mz'] < 99)[cfg.TARGETS].mean()

# valid_sample_features.groupby(valid_sample_features['sample_max_mz'] < 99)[cfg.TARGETS].mean()

# train_mass_under99 = train_sample_features[train_sample_features['sample_max_mz'] < 99].copy()

# # none of the max smple < 99 is samtestbed, so we can conclude that is right
# # for the test set, there is 7 observations that are sam testbed which are less than 99

# # it makes sense that if the molecular ion is less than 99 is because the molecule has a small mass
# # so it can not be a molecule with higher mass

# # for the train set and valid set it is the same, chloride can not have a mass of less than 99

# train_sample_features.groupby(train_sample_features['sample_min_mz'] > 1)['is_sam_testbed'].sum()
# # all of them are from sam testbed, so we can conclude anything

# train_sample_features.groupby(train_sample_features['sample_min_mz'] > 1)['is_sam_testbed'].mean()

# test_sample_features.groupby(test_sample_features['sample_min_mz'] > 1)['is_sam_testbed'].sum()
# # all of them are from sam testbed, so we can conclude anything
# # ok, any of the sam tesbed include hydrogen, whyyyyyy?

# (train_sample_features['sample_min_mz'] > 1).agg(('sum', 'mean'))

# (test_sample_features['sample_min_mz'] > 1).agg(('sum', 'mean'))

# test_sample_features['is_sam_testbed'].mean() # ok